In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

from VGGNet import VGGNet
from ResNet import ResNet
from GoogleNet import GoogleNet
from torchsummary import summary

In [2]:
lr = 1e-4
epoch = 50
batch_size = 64
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = models.resnet50(num_classes=10).to(device)#GoogleNet(aux_logits=True, num_classes=10).to(device)
googlenet = False
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr)

In [3]:
summary(model, input_size=(3, 224, 224))
#print(model)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [5]:
train_set = datasets.CIFAR10(root="/home/user/Data", train=True, transform=transform, download=True)
test_set = datasets.CIFAR10(root="/home/user/Data", train=False, transform=transform, download=True)

train_loader = DataLoader(train_set, batch_size, shuffle=True, num_workers=2, drop_last=True)
test_loader = DataLoader(test_set, batch_size, shuffle=False, num_workers=2, drop_last=True)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
for epoch_index in range(epoch):
    for data_index, [image, label] in enumerate(loader := tqdm(train_loader)):
        image = image.to(device)
        label = label.to(device)

        optimizer.zero_grad()

        if googlenet:
            aux1, aux2, output = model(image)
            aux1_loss = criterion(aux1, label)
            aux2_loss = criterion(aux2, label)
        else:
            output = model(image)

        out_loss = criterion(output, label)

        if googlenet:
            loss = out_loss + aux1_loss * 0.3 + aux2_loss * 0.3
        else:
            loss = out_loss
            
        loss.backward()
        optimizer.step()

        loader.set_postfix({"epoch": epoch_index + 1, "loss": loss.item()})

 10%|█         | 81/781 [00:35<04:58,  2.35it/s, epoch=1, loss=2.01]

In [ ]:
# 평가
correct = 0
total = 0

model.eval()

with torch.no_grad():
    for image, label in test_loader:
        image = image.to(device)
        label = label.to(device)
        
        output = model.forward(image)
        _, output_index = torch.max(output, 1)
        
        total += label.size(0)        
        correct += (output_index == label).sum().float()
        
print("Accuracy of Test Data: {}%".format(100 * correct / total))